In [1]:
from xshinnosuke.layers import Dense, Flatten, Conv2D, MaxPooling2D, AvgPooling2D, BatchNormalization, ReLU
from xshinnosuke.models import Module, Sequential
from xshinnosuke.utils import DataSet, DataLoader
from xshinnosuke.nn import Variable, CrossEntropy
from xshinnosuke.nn.optimizers import SGD
import cupy as np

In [2]:
class BasicBlock(Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__(Module)
        self.conv1 = Sequential(
            Conv2D(out_channels, 3, stride=stride, padding=1),
            BatchNormalization(),
            ReLU(True),
            Conv2D(out_channels, 3, stride=1, padding=1),
            BatchNormalization(),
        )
        if stride != 1 or in_channels != out_channels:
            self.shortcut = Sequential(
                Conv2D(out_channels, 1, stride=stride),
                BatchNormalization()
            )
        else:
            self.shortcut = Sequential()
        self.relu = ReLU(inplace=True)

    def forward(self, x, *args):
        out = self.conv1(x)
        out += self.shortcut(x)
        out = self.relu(out)
        return out



class ResNet18(Module):
    def __init__(self):
        super().__init__(Module)
        self.conv1 = Sequential(
            Conv2D(out_channels=64, kernel_size=7, stride=2, padding=3),
            BatchNormalization(),
            ReLU(inplace=True)
        )
        self.pool1 = MaxPooling2D(kernel_size=3, stride=2, padding=1)
        self.in_channels = 64

        self.layer1 = self.make_layer(BasicBlock, 64, 2, 1)
        self.layer2 = self.make_layer(BasicBlock, 128, 2, 2)
        self.layer3 = self.make_layer(BasicBlock, 256, 2, 2)
        self.layer4 = self.make_layer(BasicBlock, 512, 2, 2)
        self.pool2 = AvgPooling2D(2)
        self.flat = Flatten()
        self.fc = Dense(100)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, channels, stride))
            self.in_channels = channels
        return Sequential(*layers)


    def forward(self, x, *args):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool2(x)
        x = self.flat(x)
        x = self.fc(x)
        return x


In [3]:
# random generate data
x = np.random.rand(500, 3, 56, 56)
y = np.random.randint(0, 100, (500,))

In [4]:
net = ResNet18()
EPOCH = 5
train_data = DataSet(x, y)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
optimizer = SGD(net.parameters(), lr=0.1)
criterion = CrossEntropy()

for epoch in range(EPOCH):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = net(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        acc, loss_val = criterion.metric(pred, y)
        print('epoch %d, acc -> %f, loss -> %f' % (epoch, acc, loss_val))

epoch 0, acc -> 0.000000, loss -> 5.625706
epoch 0, acc -> 0.000000, loss -> 6.058135
epoch 0, acc -> 0.000000, loss -> 5.353103
epoch 0, acc -> 0.031250, loss -> 5.582827
epoch 0, acc -> 0.000000, loss -> 6.409279
epoch 0, acc -> 0.000000, loss -> 5.888948
epoch 0, acc -> 0.031250, loss -> 5.755154
epoch 0, acc -> 0.000000, loss -> 5.749066
epoch 0, acc -> 0.093750, loss -> 5.339854
epoch 0, acc -> 0.031250, loss -> 5.187704
epoch 0, acc -> 0.000000, loss -> 5.618297
epoch 0, acc -> 0.000000, loss -> 5.654584
epoch 0, acc -> 0.000000, loss -> 5.364009
epoch 0, acc -> 0.000000, loss -> 5.426807
epoch 0, acc -> 0.000000, loss -> 5.444934
epoch 0, acc -> 0.050000, loss -> 5.373839
epoch 1, acc -> 0.062500, loss -> 4.307467
epoch 1, acc -> 0.125000, loss -> 4.316749
epoch 1, acc -> 0.156250, loss -> 3.571934
epoch 1, acc -> 0.031250, loss -> 3.790687
epoch 1, acc -> 0.031250, loss -> 4.182666
epoch 1, acc -> 0.125000, loss -> 4.023108
epoch 1, acc -> 0.062500, loss -> 4.217873
epoch 1, ac